# Практическое задание 1

# Ранжирование вопросов StackOverflow с помощью векторных представлений слов

## курс "Математические методы анализа текстов"


### ФИО: Христолюбов Максим Евгеньевич

## Введение

В этом задании вы научитесь вычислять близость текстов и применить этот метод для поиска похожих вопросов на [StackOverflow](https://stackoverflow.com).

### Используемые библиотеки

В данном задании потребуются следующие библиотеки:
- [Gensim](https://radimrehurek.com/gensim/) — инструмент для решения различных задач NLP (тематическое моделирование, представление текстов, ...).
- [Numpy](http://www.numpy.org) — библиотека для научных вычислений.
- [scikit-learn](http://scikit-learn.org/stable/index.html) — библилиотека с многими реализованными алгоритмами машинного обучения для анализа данных.
- [Nltk](http://www.nltk.org) — инструмент для работы с естественными языками.


### Данные

Данные лежат в архиве `StackOverflowData.zip`, который состоит из:
- `train.tsv` - обучающая выборка. В каждой строке через табуляцию записаны дублирующие друг друга предложения;
- `test.tsv` - тестовая выборка. В каждой строке через табуляцию записаны: *<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...*

Скачать архив можно здесь: [ссылка на google диск](https://drive.google.com/open?id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_)

### Вектора слов

Для решения вам потребуются предобученная модель векторных представлений слов. Используйте [модель эмбеддингов](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit), которая была обучена с помощью пакета word2vec на данных Google News (100 миллиардов слов). Модель содержит 300-мерные вектора для 3 миллионов слов и фраз. Вы можете скачать их, запустив блок кода ниже.

In [1]:
# Download Google vectors to directory *target_dir*

from download_utils import download_google_vectors
download_google_vectors(target_dir='.')

## Часть 1. Предобученные векторные представления слов (2 балла)

Скачайте предобученные вектора и загрузите их с помощью функции [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) библиотеки Gensim с параметром *binary=True*. Если суммарный размер векторов больше, чем доступная память, то вы можете загрузите только часть векторов, указав параметр *limit* (рекомендуемое значение: 500000).

In [1]:
import gensim
from gensim.models import KeyedVectors

In [2]:
wv_embeddings = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


### Как пользоваться этими векторами?

Как только вы загрузите векторные представления слов в память, убедитесь, что имеете к ним доступ. Сначала вы можете проверить, содержится ли какое-то слово в загруженных эмбедингах:

    'word' in wv_embeddings

Затем, чтобы получить соответствующий вектор, вы можете использовать оператор доступа по ключу:

    wv_embeddings['word']

### Проверим, корректны ли векторные представления

Чтобы предотвратить возможные ошибки во время первого этапа, можно проверить, что загруженные вектора корректны. Для этого вы можете запустить функцию *check_embeddings*. Она запускает 3 теста:
1. Находит наиболее похожие слова для заданных "положительных" и "отрицательных" слов.
2. Находит, какое слово из заданного списка не встречается с остальными.
3. Находит наиболее похожее слово для заданного.

In [3]:
import sys
sys.path.append('/content')

from tests import check_embeddings
print(check_embeddings(wv_embeddings))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


These embeddings look good.


/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


### Векторные представления текста

Чтобы перейти от отдельных слов к векторным представлениям вопросов, предлагается подсчитать **среднее** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектоора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор. 

**<font color='red'>Задание. Реализуйте функцию *question_to_vec_by_mean*, работающую по такой логике. </font>**

In [3]:
import numpy as np
import string

In [4]:
def question_to_vec_by_mean(question, embeddings, dim=300):
  words = question.split(' ')
  words = [word for word in words if word != ""]
  res, num = np.zeros(dim), 0
  for word in words:
    if word in wv_embeddings:
      num += 1
      res = res + embeddings[word]
  num = 1 if num == 0 else num
  return res/num

Для базовой проверки решения запустите клетку ниже.

In [5]:
from tests import question_to_vec_tests
print(question_to_vec_tests(question_to_vec_by_mean, wv_embeddings))

Basic tests are passed.


Теперь у нас есть метод для создания векторного представления любого предложения. Оценим, как будет работать это решение.

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из *N* вопросов *R* случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели *R + 1* примеров и смотреть на позицию дубликата.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то *K*:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)],$$
где $q_i$ - $i$-ый вопрос, $dup_i$ - его дубликат, $topK(q_i)$ - первые *K* элементов в ранжированном списке, который выдает наша модель.

#### DCG@K
Второй метрикой будет упрощенная [DCG метрика](https://en.wikipedia.org/wiki/Discounted_cumulative_gain):
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le K],$$
где $rank_{dup_i}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$. С такой метрикой модель штрафуется за низкую позицию корректного ответа.

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. Пусть $N = 1$, $R = 3$, вопрос $q_1$ это "Что такое python", а его дубликат $dup_1$ это "Что такое язык python". Пусть модель выдала следующий ранжированный список кандидатов:

1. *"Как узнать с++"*
2. *"Что такое язык python"*
3. *"Хочу учить Java"*
4. *"Не понимаю Tensorflow"*

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [dup_1 \in top1(q_1)] = 0$
- [K = 4] $\text{Hits@4} =  [dup_1 \in top4(q_1)] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

**<font color='red'>Задание. Реализуйте функции *hits_count* и *dcg_score*. </font>**

Каждая функция имеет два аргумента: *dup_ranks* и *k*. *dup_ranks* является списком, который содржит *рейтинги дубликатов* (их позиции в ранжированном списке). Например, *dup_ranks = [2]* для примера, описанного выше.

In [5]:
def hits_count(dup_ranks, k):
  return sum([el <= k for el in dup_ranks])/len(dup_ranks)

In [6]:
def dcg_score(dup_ranks, k):
  dup_ranks_tmp = list(filter(lambda x: x <= k, dup_ranks))
  return sum([1/np.log2(1+el) for el in dup_ranks_tmp])/len(dup_ranks)

Протестируйте функции. Успешное прохождение базовых тестов еще не гарантирует корректности реализации!

In [8]:
from tests import test_hits
print(test_hits(hits_count))

Basic test are passed.


In [9]:
from tests import test_dcg
print(test_dcg(dcg_score))

Basic test are passed.


### Ранжирование вопросов StackOverflow

Выборка уже разбита на обучающую и тестовую. Все файлы используют табуляцию в качестве разделителя, но они имеют разный формат:

- *обучающая* выборка (test.tsv) содержит похожие друг на друга предложения в каждой строке;
- *тестовая* выборка (validation.tsv) содержит в каждой строке: *вопрос, похожий вопрос, отрицательный пример 1, отрицательный пример 2, ...*

Считайте тестовую выборку для оценки качества текущего решения.

In [7]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

In [8]:
validation = read_corpus("validation.tsv")

<font color='red'>**Задание. Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния.**</font>
    
Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список *[(2, c), (0, a), (1, b)]*.

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
def rank_candidates(question, candidates, embeddings, dim=300):
  vec_question = question_to_vec_by_mean(question, embeddings, 300).reshape(1, -1)  
  scores = [(candidate, cosine_similarity(vec_question, question_to_vec_by_mean(candidate, embeddings, 300).reshape(1, -1))[0,0]) for candidate in candidates]
  scores = enumerate(scores)
  pairs = sorted(scores, reverse=True, key = lambda x: x[1][1])
  return [(i, x[0]) for (i, x) in pairs]

Протестируйте работу функции на примерах ниже.

In [14]:
from tests import test_rank_candidates
print(test_rank_candidates(rank_candidates, wv_embeddings))

Basic tests are passed.


Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояний между векторами в случае неэффективной реализации может занимать до 10 минут, разумнее сделать векторную реализацию.

In [ ]:
wv_ranking = []
for line in tqdm(validation):
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.217 | Hits@   1: 0.217
DCG@   5: 0.271 | Hits@   5: 0.318
DCG@  10: 0.288 | Hits@  10: 0.372
DCG@ 100: 0.328 | Hits@ 100: 0.571
DCG@ 500: 0.360 | Hits@ 500: 0.827
DCG@1000: 0.379 | Hits@1000: 1.000


Если вы проделали все шаги правильно, то вы должны разочароваться полученными результатами. Давайте попробуем понять, почему качество модели такое низкое. Когда вы работаете с какими-либо данными, очень полезно первым делом посмотреть на них глазами. Выведите несколько вопросов из наших данных:

### Предобработка текстов

Как вы могли заметить, мы имеем дело с сырыми данными. Это означает, что там присутствует много опечаток, спецсимволов и заглавных букв. В нашем случае это все может привести к ситуации, когда для данных токенов нет предобученных векторов. Поэтому необходима предобработка.

<font color='red'>**Задание. Реализуйте функцию предобработки текстов.**</font>

Вам требуется:
- Перевести символы в нижний регистр;
- Заменить символы пунктуации на пробелы;
- Удалить "плохие" символы;
- Удалить стопслова.

In [11]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
def text_prepare(text):
  text = text.lower()

  for c in string.punctuation:
    text = text.replace(c, " ")

  text = re.sub('[^a-zA-Z]+', ' ', text)

  words = text.split(' ')
  words = [word for word in words if word != ""]

  sw = set(stopwords.words('english'))

  res_words = []
  for w in words:
    if w not in sw:
      res_words.append(w)
  text = ' '.join(res_words)

  return(text)

<font color='red'>**Задание. Теперь преобразуйте все вопросы из тестовой выборки. Оцените, как изменилось качество. Сделайте выводы.**</font>

In [13]:
for i, line in enumerate(validation):
  validation[i] = [text_prepare(q) for q in line]

In [ ]:
wv_ranking = []
for line in tqdm(validation):
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

del wv_ranking

DCG@   1: 0.357 | Hits@   1: 0.357
DCG@   5: 0.428 | Hits@   5: 0.493
DCG@  10: 0.444 | Hits@  10: 0.542
DCG@ 100: 0.479 | Hits@ 100: 0.717
DCG@ 500: 0.498 | Hits@ 500: 0.869
DCG@1000: 0.512 | Hits@1000: 1.000


При предобработке данных малоинформативные символы и слова удаляются, что уменьшает расстояние между схожими вопросами и увеличивает между разными вопросами. С другой стороны регистр теперь не имеет значения, что приводит к использованию одного эмбединга для слова при любых регистрах, что имеет смысл для повышения точности. Кроме того некоторые слова могли вообще не находится в словаре из за различия в регистрах.

## Часть 2. Представления для неизвестных слов. (3 балла)

<font color='red'>**Задание. Оцените долю слов в выборке, для которых нет эмбеддинга в модели.**</font>

In [ ]:
def get_all_words(questions):
  all_known_words = []
  for line in questions:
    for q in line:
      words = q.split(' ')
      all_known_words += words
  return all_known_words

all_words = get_all_words(validation[:2000])

n = 0
for word in all_words:
    n += not word in wv_embeddings

print("Доля слов без эмбединга", n/len(all_words))
del all_words

Доля слов без эмбединга 0.10223730226366186


Для того, что получить представления для неизвестного слова, воспользуемся следующим подходом:
    
1. Будем восстанавливать эмбеддинг неизвестного слова как сумму эмбеддингов буквенных триграмм. Например, слово where должно представляться суммой триграмм _wh, whe, her, ere, re_

2. В качестве обучающих данных будем использовать слова, для которых есть эмбеддинг в модели. Будем обучать эмбеддинги триграмм по выборке эмбеддингов с помощью функционала MSE:

$$L = \sum_{w \in W_{known}}\| f_{\theta}(w) - v_w \|^2 \to \min_{\theta}$$

где:

* $W_{known}$ — множество известных модели слов
* $f_{\theta}(w)$ — сумма эмбеддингов триграмм слова $w$
* $v_w$ — эмбеддинг слова $w$
* $\theta$ — веса эмбеддингов триграмм

<font color='red'>**Задание. Реализуйте предложенную модель ниже.**</font>

Используйте класс nn.EmbeddingBag для построения среднего вектора представлений.

In [14]:
from tqdm.notebook import tqdm
from collections import Counter
import torch
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import re

In [19]:
class EmbeddingDataset(Dataset):
    def __init__(self, indexes_of_trigrams, words_embeddings):
      self.indexes_of_trigrams = indexes_of_trigrams
      self.words_embeddings = words_embeddings

    def __getitem__(self, i):
        return self.indexes_of_trigrams[i], self.words_embeddings[i]
        
    def __len__(self):
        return len(self.words_embeddings)

In [20]:
def get_index_change_len(data_sort):
  index_change_len = []
  len_prev_indexs = 0
  for i, (indexs, embeding)  in enumerate(data_sort):
    len_indexs = len(indexs)
    if len(indexs) > len_prev_indexs:
      index_change_len.append(i)
      len_prev_indexs = len_indexs
  index_change_len.append(len(data_sort))
  return index_change_len

def word_to_trigrams(word):
    if len(word) < 3:
        return [word]
    trigrams = re.findall(r'(?=([a-zA-Z]{3}))', word)
    trigrams.append(word[:2])
    trigrams.append(word[-2:])
    return trigrams

def create_datasets(data):
  data_sort = sorted(data, key=lambda tup: len(tup[0]))
  index_change_len = get_index_change_len(data_sort)
  datasets = []
  for i, ind in tqdm(enumerate(index_change_len)):
    if i < 30:
      collection = [el[0] for el in data_sort[0 : index_change_len[i+1] - ind]]
      target = [el[1] for el in data_sort[0 : index_change_len[i+1] - ind]]
      datasets.append(EmbeddingDataset(torch.LongTensor(collection), torch.FloatTensor(target)))        
      del data_sort[0 : index_change_len[i+1] - ind], collection, target
  return datasets

In [21]:
trigram_to_index = Counter()
all_words = set()
for word in tqdm(wv_embeddings.vocab.keys()):
  if word == word.lower() and word == word.replace('_', ' '):
    all_words.add(word)
for line in tqdm(validation):
  for quest in line:
    words = quest.split(' ')
    for word in words:
      if word == word.lower() and word == word.replace('_', ' ') and word !='':
        all_words.add(word)

data = []
trigram_to_index = {}
for word in tqdm(all_words):
  current_word_trigrams = []
  for trigram in word_to_trigrams(word):
      trigram_to_index.setdefault(trigram, len(trigram_to_index))
      current_word_trigrams.append(trigram_to_index[trigram])
  if word in wv_embeddings:
    data.append((current_word_trigrams, wv_embeddings[word]))

del all_words

In [23]:
datasets = create_datasets(data)
data_loaders = []
for dataset in datasets:
  data_loaders.append(DataLoader(dataset, batch_size=1024, shuffle=True, num_workers = 4))

del data

In [22]:
class TrigrammEmbeddingsModel(nn.Module):
    def __init__(self, len_tokens, embedding_dim=300):
        super(TrigrammEmbeddingsModel, self).__init__()

        self.embedding = nn.Embedding(len_tokens, embedding_dim)
        self.embedding.weight.data.uniform_(-0.5, 0.5)
        
    def forward(self, tokens):    # токен-слово = массив индексов его триграмм
        return torch.sum(self.embedding(tokens), 1)

<font color='red'>**Задание. Обучите модель. Оцените, как изменилось качество. Сделайте выводы.**</font>

Если вы всё реализовали правильно, качество решения должно вырасти.

In [25]:
device = 'cuda'

In [26]:
trigramm_model = TrigrammEmbeddingsModel(len(trigram_to_index))
trigramm_model = trigramm_model.to(device)

loss_function = nn.MSELoss()
optimizer = Adam(trigramm_model.parameters(), lr=1e-3, weight_decay = 1e-7)

In [27]:
def train(trigramm_model, data_loaders, datasets, loss_function, optimizer, epochs = 20):
  len_data = sum([dataset.words_embeddings.shape[0] for dataset in datasets])

  all_losses = []
  for n_epoch in tqdm(range(epochs)):
    mean_loss = 0
    for data_loader in data_loaders:
      for trigrams_indexes, embedding_word in data_loader:
        trigrams_indexes = trigrams_indexes.to(device)
        embedding_word = embedding_word.to(device)

        new_embedding = trigramm_model(trigrams_indexes)

        loss_value = loss_function(new_embedding, embedding_word)
        loss_value.backward()

        mean_loss += float(loss_value.to('cpu'))
        optimizer.step()
        optimizer.zero_grad()
    mean_loss /= len_data
    print(n_epoch, mean_loss)
    all_losses.append(mean_loss)
  return all_losses

In [28]:
all_losses = train(trigramm_model, data_loaders, datasets, loss_function, optimizer, epochs = 30)
torch.save(trigramm_model.state_dict(), 'model.ckpt')

del datasets, data_loaders

0 0.0004447210884648536
1 0.00022277627027249855
2 0.0001419136190914711
3 9.870867383780049e-05
4 7.352300373488756e-05
5 5.783035484712467e-05
6 4.758082207045252e-05
7 4.063562461184745e-05
8 3.57699633616442e-05
9 3.227133798817715e-05
10 2.9725932384812435e-05
11 2.782756888855493e-05
12 2.642388752697741e-05
13 2.536440440120893e-05
14 2.4543673371864045e-05
15 2.390880941829031e-05
16 2.3389977029496535e-05
17 2.3013678887430916e-05
18 2.2687807473304232e-05
19 2.2472181109598043e-05
20 2.226640534403512e-05
21 2.2103006644536414e-05
22 2.198479706171062e-05
23 2.1851325717487376e-05
24 2.1774139262619898e-05
25 2.1727483596104256e-05
26 2.1668637746754474e-05
27 2.1639209038520428e-05
28 2.1595559948638095e-05
29 2.1541404848347676e-05



In [18]:
trigramm_model = TrigrammEmbeddingsModel(len(trigram_to_index))
trigramm_model.load_state_dict(torch.load('model.ckpt'))
trigramm_model.eval()

NameError: ignored

In [24]:
def question_to_vec_by_mean(question, embeddings, dim=300):
  words = question.split(' ')
  words = [word for word in words if word != ""]
  res = np.zeros(dim)
  for word in words:
    if word in wv_embeddings:
      res = res + embeddings[word]
    else:
      trigrams = word_to_trigrams(word)
      if all(trigram in trigram_to_index for trigram in trigrams):
        indexes = [trigram_to_index[trigram] for trigram in trigrams]
        res = res + trigramm_model(torch.LongTensor([indexes])).detach().numpy()[0]
  num = 1 if len(words) == 0 else len(words)
  return res/num

In [31]:
wv_ranking = []
for line in tqdm(validation):
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))


DCG@   1: 0.364 | Hits@   1: 0.364
DCG@   5: 0.439 | Hits@   5: 0.504
DCG@  10: 0.456 | Hits@  10: 0.557
DCG@ 100: 0.492 | Hits@ 100: 0.737
DCG@ 500: 0.512 | Hits@ 500: 0.891
DCG@1000: 0.524 | Hits@1000: 1.000


Качество возрастает, но всего на $3\%$. Вообще, такой подход дает возможность в некоторой степени бороться с опечатками и словообразованием, для языков подобных русскому этот подход более разумен, чем к английскому вследствие богатого словообразования.

## Часть 3. Обучение метрики. (5 баллов)

Косинусное расстояние на фиксированных предобученных векторах - самое простое, но не самое лучшее решение. 
Качество можно улучшить, если обучить функцию близости для решения конкретной задачи.
    
В этом пункте вам предлагается обучить полносвязную нейронную сеть, предсказывающую близость пары вопросов в диапазоне от 0 до 1. Для этого предлагается использовать следующую нейронную сеть:

- вход: векторные представления пары вопросов (нескольких пар, если использовать обучение по батчам)
- выход: ненормированное число, показатель близости вопросов
- архитектура: сначала необходимо сагрегировать эмбеддинги пар (например, сконкатенировать), а затем применить несколько полносвязных слоёв (рекомендуется 2) с нелинейностями (например, `torch.nn.ReLU`)
- функция потерь: кросс-энтропия от сигмоиды выхода сети (рекомендуется использовать `torch.nn.BCELoss``)

Пример архитектуры показан на картинке ниже:
<img src="dssm_we_problem.png" alt="dssm"
	title="nn example" width="300" height="300" />


Чтобы учить такую модель, нужны как и положительные примеры (дубликаты вопросов), так и отрицательные (вопросы, которые не являются дубликатами). Важное значение имеет метод выбора отрицательных примеров. Самый базовый вариант - на каждую верную пару (s1, s2) случайно выбирать 5-10 случайных пар (s1, s).
Более сложная стратегия - использовать в качестве примеров чем-то похожие, но на самом деле не близкие пары вопросов (например, можно использовать для подбора примеров уже опробованный выше метод через косинусное расстояние).

Обучите нейросеть (на обучающих данных) и посчитайте с её помощью заданные метрики на тестовых данных. Оценка за этот пункт будет зависеть от итоговых значений метрики (ожидается значение Hits@ 1 не менее 0.53). Для того, чтобы добиться этого, можно воспользоваться следующими идеями:
- для обучения сети достаточно 5-12 эпох
- для такого рода задач хорошей агрегацией входных векторов u и v является конкатенированный вектор \[u, v, u - v, u * v\], где все операции поэлементные
- если 2 слоя вам мало, попробуйте 3-5 и различное число нейронов
- исходные векторные представления можно дообучать, можно делать это сразу, но лучше сперва заморозить их веса, а затем разморозить после нескольких эпох

- больший объём обучающих данных может ощутимо повысить качество
- ещё качество обычно растёт при правильном использовании регуляризации (например, `torch.nn.Dropout`, `torch.nn.BatchNorm1D`)

__За задание можно получить до 2 бонусных баллов, если значение Hits@ 1 превысит на валидации 0.55 (балл будет зависитеть от величины превышения).__

<font color='red'>**Опишите модель и функцию генерации примеров. Обучите модель, оцените, как изменилось качество. Сделайте выводы.**</font>

In [19]:
class QuestionsDataset(Dataset):  # k соотношение между положительными и отрицательными примерами
    def __init__(self, filename, num_quest, k):
        train = read_corpus(filename)
        pos_pairs = train[:num_quest+k]

        for i, line in tqdm(enumerate(pos_pairs)):
            pos_pairs[i] = [question_to_vec_by_mean(text_prepare(quest), wv_embeddings) for i, quest in enumerate(line[:2])] 

        pos_pairs = torch.FloatTensor(pos_pairs)
        neg_pairs = torch.FloatTensor(pos_pairs.shape[0]*k, pos_pairs.shape[1], pos_pairs.shape[2])
        
        for i in range(num_quest):
          for j in range(k):
            neg_pairs[k*i+j, 0, :] = pos_pairs[i, 0, :] 
            neg_pairs[k*i+j, 1, :] = pos_pairs[i+j+1, 1, :] 
        self.pairs = torch.cat((pos_pairs, neg_pairs), dim=0)
        self.targets = torch.cat((torch.ones(len(pos_pairs)), torch.zeros(len(neg_pairs))), dim=0)

    def __getitem__(self, i):
        return self.pairs[i], self.targets[i]
        
    def __len__(self):
        return len(self.targets)

In [20]:
class QuestionsDatasetValidation(Dataset):  # k соотношение между положительными и отрицательными примерами
    def __init__(self, filename, num_quest, k):
        train = read_corpus(filename)
        pos_pairs = train[:num_quest+k]

        for i, line in tqdm(enumerate(pos_pairs)):
            pos_pairs[i] = [question_to_vec_by_mean(text_prepare(quest), wv_embeddings) for i, quest in enumerate(line[:k+1])] 

        pos_pairs = torch.FloatTensor(pos_pairs)
        neg_pairs = torch.FloatTensor(pos_pairs.shape[0]*k, pos_pairs.shape[1], pos_pairs.shape[2])
        
        for i in tqdm(range(num_quest)):
          for j in range(k):
            neg_pairs[k*i+j, 0, :] = pos_pairs[i, 0, :] 
            neg_pairs[k*i+j, 1, :] = pos_pairs[i, 1+j, :] 
        self.pairs = torch.cat((pos_pairs, neg_pairs), dim=0)
        self.targets = torch.cat((torch.ones(len(pos_pairs)), torch.zeros(len(neg_pairs))), dim=0)

    def __getitem__(self, i):
        return self.pairs[i], self.targets[i]
        
    def __len__(self):
        return len(self.targets)

In [21]:
class ProximityFunctionModel(nn.Module):
    def __init__(self):
        super(ProximityFunctionModel, self).__init__()
        self.layer1 = nn.Linear(1200, 100)
        self.bn1 = nn.BatchNorm1d(num_features=100)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(100, 1)

    def forward(self, tokens1, tokens2):    
        X = torch.cat((tokens1, tokens2, tokens1 - tokens2, tokens1*tokens2), 1)
        X = self.layer1(X)
        X = self.bn1(X)
        X = self.relu(X)
        X = self.layer2(X)
        return X

In [22]:
def train(model, data_loader_train, loss_function, optimizer, epochs = 20):
  all_losses = []
  for n_epoch in tqdm(range(epochs)):
    model.train()
    tr_loss, val_loss = 0, 0
    for pairs, targets in data_loader_train:
        pairs = pairs.to(device)
        targets = targets.to(device)

        proximitys = model(pairs[:, 0, :], pairs[:, 1, :])[:, 0]

        loss_value = loss_function(proximitys, targets)
        loss_value.backward()

        tr_loss += float(loss_value.to('cpu'))
        optimizer.step()
        optimizer.zero_grad()


    print(n_epoch, tr_loss)
    all_losses.append([tr_loss])

  return all_losses

In [23]:
dataset = QuestionsDataset('train.tsv', 100000, 10)
data_loader_train = DataLoader(dataset, batch_size=1024, shuffle=True, num_workers = 4)
loss_function = torch.nn.BCEWithLogitsLoss()
prox_model = ProximityFunctionModel()

In [24]:
device = 'cuda'
prox_model = prox_model.to(device)
optimizer = Adam(prox_model.parameters(), lr=1e-3, weight_decay = 1e-4)

In [26]:
all_losses = train(prox_model, data_loader_train, loss_function, optimizer, epochs = 30)
torch.save(prox_model.state_dict(), 'prox.ckpt')
del data_loader_train, dataset

0 356.7684903740883
1 355.37730517983437
2 355.60995802283287
3 355.0372861921787
4 354.35163763165474
5 354.4004729986191
6 354.5293194055557
7 354.5512338876724
8 354.69487059116364
9 353.4831696152687
10 353.33881852030754
11 353.2217388153076
12 353.1441289484501
13 352.79965859651566
14 352.673216342926
15 353.2461479008198
16 352.35881343483925
17 352.86201667785645
18 352.19966104626656
19 352.95339062809944
20 352.9293181300163
21 351.68663677573204
22 352.53074955940247
23 351.91834753751755
24 351.7586619257927
25 351.9749391376972
26 351.9757954776287
27 352.1462379693985
28 351.8522963821888
29 351.6481333374977



In [31]:
prox_model.eval()

ProximityFunctionModel(
  (layer1): Linear(in_features=1200, out_features=100, bias=True)
  (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (layer2): Linear(in_features=100, out_features=1, bias=True)
  (drop1): Dropout(p=0.3, inplace=False)
)

In [32]:
sigmoid = nn.Sigmoid()
prox = lambda v1, v2: sigmoid(prox_model(torch.FloatTensor(v1), torch.FloatTensor(v2)))

def rank_candidates(question, candidates, embeddings, dim=300):
  vec_question = question_to_vec_by_mean(question, embeddings, 300).reshape(1, -1)  
  scores = [(candidate, prox(vec_question, question_to_vec_by_mean(candidate, embeddings, 300).reshape(1, -1))[0,0]) for candidate in candidates]
  scores = enumerate(scores)
  pairs = sorted(scores, reverse=True, key = lambda x: x[1][1])
  return [(i, x[0]) for (i, x) in pairs]

In [34]:
wv_ranking = []
for line in tqdm(validation):
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

KeyboardInterrupt: ignored

## Бонусная часть (3 балла)

Такую же модель можно обучать с помощью Softmax Loss. Попробуйте изменить процесс обучения следующим образом:

- для каждой правильной пары $(s_1, s_2)$ генерируются отрицательные примеры с первым вопросом $s_1$
- для всех пар через нейросеть считается близость, вектор близостей поступает в $softmax$, после этого мы получаем вероятности $p(s_i|s_1)$
    
    Например, для 100 негативных примеров и нейросети, вычисляющей $sim$:
    
    $p(s_{i}|s_{1}) = softmax_{i \in \{2, 3, \ldots, 100\}} sim(s_i, s_1)$


- функция потерь: кросс-энтропия, максимизируем вероятность правильного примера $s_2$ при условии $s_1$

<font color='red'>**Реализуйте функции генерации батчей и обучения модели для указанного типа лосса. Сделайте выводы.**</font>

In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

Процедура вычисления $softmax$ на больших массивах может быть затратной по времени, попробуем её ускорить. [Пример](https://discuss.pytorch.org/t/feedback-on-manually-implemented-hierarchical-softmax/82478) реализации, от которого можно отталкиваться.

<font color='red'>**Измерьте время, затрачиваемое процессом обучения на подсчёт лосс-функционала. Реализуйте модуль иерархического $softmax$, который будет на этапе обучения заменять обычный $softmax$. Сравните затрачиваемое на его подсчёт время с обычной реализацией $softmax$. Сделайте выводы.**</font>

In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################